In [ ]:
<center>
    <div id="field">
        <h2>Soon here</h2>
        <h1>John Conway's game of life!</h1>
    </div>
    <div id="monitor">
    <h2>
        Iteration <output name="iteration">0</output> - <output name="cellCount">0</output> live cells
    <h2>
        Generated in <output name="generationDuration">0</output> ms - rendered in <output name="renderDuration">0</output> ms
    <div>
</center>

In [ ]:
// update element IDed field
updateField = function(html) {
    field.innerHTML = html;
};

// update named elements with a dictionary { nom d'élément: valeur } 
updateMonitor = function(dict) {
    for (const key of Object.keys(dict)) {
        document.getElementsByName(key)[0].value = dict[key];
    }
}

In [ ]:
using System.Collections.Immutable;

record Cell(int X, int Y)
{
    public Cell[] Neighbors =>
        new[] 
        {
            new Cell(X + 1, Y + 1), new Cell(X, Y + 1), new Cell(X - 1, Y + 1), 
            new Cell(X + 1, Y),                         new Cell(X - 1, Y),     
            new Cell(X + 1, Y - 1), new Cell(X, Y - 1), new Cell(X - 1, Y - 1), 
        };
}

record Universe(int Iteration, ImmutableHashSet<Cell> LiveCells)
{
    /// logique du jeu de la vie
    public Universe NextGeneration()
    {                                               // [[[
         var nextCells = LiveCells                  //  L'univers est représenté par un inventaire des cellules vivantes.
            .SelectMany(cell => cell.Neighbors)     //  A chaque génération, on calcule le nouvel état des cellules vivantes,
            .Concat(LiveCells)                      //  et de leurs voisines (vivantes ou vides). Le type choisi est un hashset,
            .Distinct()                             //  car il est soumis à des vérifications de contenance très fréquent.
            .Where(cell =>                          // ]]]
                {   
                    var isAlive = LiveCells.Contains(cell);
                    var liveNeighbors = LiveCells.Intersect(cell.Neighbors);
                    return (isAlive, liveNeighbors.Count) switch
                    {
                        (true, 2 or 3) => true,     // Une cellule reste vivante si elle a 2 ou 3 voisines.
                        (false, 3) => true,         // Une cellule devient vivante si elle a 3 voisines.
                        (_, _) => false             // Autrement, elle meurt ou reste vide.
                    };
                })
            .ToImmutableHashSet();
        return new Universe(Iteration + 1, nextCells);
    }

    /// Extrait une univers d'une chaîne multiligne représentant une matrice où les O sont des cellules vivantes.
    public static Universe BuildFromText(string text) 
    {
        var cells = text.Split('\n')
            .SelectMany((line, i) => line.Select((chr, j) => (chr, i, j)))
            .Where(x => x.chr == 'O')
            .Select(x => new Cell(x.j, x.i));
        return new Universe(0, cells.ToImmutableHashSet());
    }

    /// Génère une balise html de type svg représentant graphiquement l'univers.
    public string ToSvgMarkup(int width, int height, int viewSize)
    {
        var sb = new System.Text.StringBuilder();
        sb.AppendLine(@$"<svg width=""{viewSize}"" viewBox=""0 0 {width} {height}"">");
        sb.AppendLine(@$"<rect width=""{width}"" height=""{height}"" fill=""#222222"" />");
        foreach (var cell in LiveCells)
        {
            sb.AppendLine(@$"<rect width=""0.98"" height=""0.98"" x=""{cell.X}"" y=""{cell.Y}"" rx=""0.30"" fill=""LightBlue""></rect>");
        }
        sb.AppendLine("</svg>");
        return sb.ToString();
    }

    /// Génère aléatoirement un univers
    public static Universe BuildRandom(int width, int height, int percentageAlive)
    {
        var rng = new Random();
        var cells = ImmutableHashSet.Create<Cell>();
        for (int i = 0; i < width; i++)
        {
            for (int j = 0; j < height; j++)
            {
                if (rng.NextDouble() < (double)percentageAlive / 100.0)
                cells = cells.Add(new Cell(i, j));
            }
        }
        return new Universe(0, cells);
    }
}

In [ ]:
#!value --name gosperGliderGun
______________________________________________
______________________________________________
______________________________________________
______________________________________________
_____________________________O________________
___________________________O_O________________
_________________OO______OO____________OO_____
________________O___O____OO____________OO_____
_____OO________O_____O___OO___________________
_____OO________O___O_OO____O_O________________
_______________O_____O_______O________________
________________O___O_________________________
_________________OO___________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________

In [ ]:
#!share gosperGliderGun --from value


using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;
using System.Diagnostics;

var universe = Universe.BuildFromText(gosperGliderGun);

await RunUniverse(universe, width: 46, height: 25);

async Task RunUniverse(Universe universe, int width, int height, int viewSize = 800)
{
    var sw = new Stopwatch();
    while (! KernelInvocationContext.Current.CancellationToken.IsCancellationRequested)
    {
        var delay = Task.Delay(120);
        sw.Restart();
        await SendJSCodeAsync($"updateField(`{universe.ToSvgMarkup(width, height, viewSize)}`);");
        await SendJSCodeAsync($@"updateMonitor({{ 
            ""cellCount"": {universe.LiveCells.Count}, 
            ""iteration"": {universe.Iteration},
            ""renderDuration"": {sw.ElapsedMilliseconds} 
        }});");
        sw.Restart();
        universe = universe.NextGeneration();
        await SendJSCodeAsync($@"updateMonitor({{ ""generationDuration"": {sw.ElapsedMilliseconds} }});");
        await delay;
    }

    async Task SendJSCodeAsync(string code) => await Kernel.Root.SendAsync(new SubmitCode(code, "javascript"));
}